In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as stats
import pylab as plt
from collections import Counter
import datetime

%matplotlib notebook

# flag to control where data is loaded to
mode = 'psycopg2'

# how many times to run each test for tracking mean/std dev

# sqlite stuff
if mode == 'sqlite3':
    import sqlite3
    conn = sqlite3.connect("../../data/sepsis.db")
    cursor = conn.cursor()
elif mode == 'psycopg2': # alternatively use postgresql
    import psycopg2
    connect_str = "dbname='sepsis' user='sepsis' host='localhost' " + \
                  "password='sepsis'"
    conn = psycopg2.connect(connect_str)
    cursor = conn.cursor()

qlog_conn = sqlite3.connect('../../data/kfp_log.db')
q_cursor = qlog_conn.cursor()

start = datetime.datetime.now()
# because names are created as case sensistive in postgres, must be quoted...
# should probably fix that...
sql = '''
SELECT d."SubjectId",
    d."EncounterId",
    d."Source",
    -- d.StartDate,
    d."Code",
    d."Type",
    MAX("FlowsheetValue") AS MaxScore,
    -- AVG("FlowsheetValue") AS MeanScore,
    MIN("FlowsheetValue") AS MinScore,
    COUNT("FlowsheetValue") AS NumLoggedScores
 FROM diagnoses d
 LEFT JOIN flowsheet f
 ON d."EncounterId" = f."EncounterId"
 GROUP BY d."SubjectId", d."EncounterId", d."Source", d."Code", d."Type"
 ORDER BY NumLoggedScores DESC
 limit
'''
# timing this query on databases
# df = pd.read_sql(sql,conn)
# sqlite - 42 to 60 seconds
# postgres - 30 seconds

#sql = 'SELECT subjectid, encounterid, source, code, type FROM "diagnoses" limit 100'


In [ ]:
# query cache
store = {}

def prefetch_query(n):
    if n not in store:
        store[n] = pd.read_sql(sql + n, conn)        
    return store[n]

In [ ]:
# sizes of patient population to evaluate
patient_population = ['10', '100', '1000', '10000', '100000']
# how many times to run test to calculate mean/std dev
default_repetitions = 1
# track each run for calculations
query_timings = []
fauxify_timings = []

def show_timings(df):
    q = pd.read_sql("SELECT * FROM kfp_log order by fauxify_end",qlog_conn)
    print('Method used     :', q.tail(1)['faux_method'].iloc[0])
    print('Time for query  :', (pd.to_datetime(q.tail(1)['query_end']) - pd.to_datetime(q.tail(1)['query_start'])).iloc[0].total_seconds())
    print('Time for fauxify:', (pd.to_datetime(q.tail(1)['fauxify_end']) - pd.to_datetime(q.tail(1)['fauxify_start'])).iloc[0].total_seconds())
    print('Size of dataset :', len(df), 'rows')

# rerun_query option doesn't time fauxify method... need to fix that
def time_method(kfpd, repetitions = default_repetitions, verbose = True, rerun_query = True):
    for n in patient_population:
        for i in range(1, repetitions + 1):
            # if dataframe provided, don't need to re-run query
            if rerun_query:
                fdf=kfpd.read_sql(sql + n,conn)
                q = pd.read_sql("SELECT * FROM kfp_log order by fauxify_end",qlog_conn)
                query_timings.append((pd.to_datetime(q.tail(1)['query_end']) - pd.to_datetime(q.tail(1)['query_start'])).iloc[0].total_seconds())
                fauxify_timings.append((pd.to_datetime(q.tail(1)['fauxify_end']) - pd.to_datetime(q.tail(1)['fauxify_start'])).iloc[0].total_seconds())
            else:
                df = prefetch_query(n)
                start = datetime.datetime.now()
                fdf=kfpd.plugin.fauxify(df)
                fauxify_timings.append((datetime.datetime.now() - start).total_seconds())
            if verbose:
                print('Iteration ', i, 'of ', repetitions)
                print('Method used              :', q.tail(1)['faux_method'].iloc[0])
                print('Size of dataset returned :', len(fdf), 'rows')
                if rerun_query:
                    print('Time for query           :', query_timings[-1])
                print('Time for fauxify         :', fauxify_timings[-1])
        print('Method used             :', type(kfpd.plugin).__name__)
        print('Size of dataset returned:', len(fdf), 'rows')
        print('    Fauxify Mean   :', np.mean(fauxify_timings))
        print('    Fauxify Std Dev:', np.std(fauxify_timings))
        if rerun_query:
            print('    Query Mean   :', np.mean(query_timings))
            print('    Query Std Dev:', np.std(query_timings))
        else:
            print('    See previous run for query timings')
    return fdf

In [ ]:
from importlib import reload
from kungfauxpandas import KungFauxPandas, TrivialPlugin, DataSynthesizerPlugin, KDEPlugin
kfpd = KungFauxPandas()

In [ ]:
kfpd.plugin = TrivialPlugin()
fdf = time_method(kfpd)
fdf.head()

In [ ]:
kfpd.plugin = TrivialPlugin()
fdf = time_method(kfpd, verbose = False, rerun_query = False)
fdf.head()

In [ ]:
kfpd.plugin = KDEPlugin(verbose = False)
time_method(kfpd, verbose = False, rerun_query = False)

### DataSynthesizer, two different methods with no configuration

In [ ]:
kfpd.plugin = DataSynthesizerPlugin(mode='correlated_attribute_mode')
for n in ['10', '100', '1000', '10000', '100000']:
    fdf=kfpd.read_sql(sql + n,conn)
    show_timings(fdf)

fdf.head()

In [ ]:
kfpd.plugin = DataSynthesizerPlugin(mode='independent_attribute_mode')
for n in ['10', '100', '1000', '10000', '100000']:
    fdf=kfpd.read_sql(sql + n,conn)
    show_timings(fdf)

fdf.head()

### Now try DataSynthesizerPlugin with some manual configuration

In [ ]:
kfpd.plugin = DataSynthesizerPlugin(mode='correlated_attribute_mode',
                                    candidate_keys = {'SubjectId': True, 'EncounterId': True},
                                    categorical_attributes = {'Source': True, 'Code': True, 'Type': True}
                                   )
for n in ['10']: #, '100', '1000', '10000', '100000']:
    fdf=kfpd.read_sql(sql + n,conn)
    show_timings(fdf)

fdf.head()

In [ ]:
kfpd.plugin = DataSynthesizerPlugin(mode='independent_attribute_mode',
                                    candidate_keys = {'SubjectId': True, 'EncounterId': True},
                                    categorical_attributes = {'Source': True, 'Code': True, 'Type': True}
                                   )
for n in ['10', '100', '1000', '10000', '100000']:
    fdf=kfpd.read_sql(sql + n,conn)
    show_timings(fdf)

fdf.head()

In [ ]:
degree_of_bayesian_network
kfpd.plugin = DataSynthesizerPlugin(mode='correlated_attribute_mode',
                                    candidate_keys = {'SubjectId': True, 'EncounterId': True},
                                    categorical_attributes = {'Source': True, 'Code': True, 'Type': True},
                                    degree_of_bayesian_network = 3 # default is 2
                                   )
for n in ['10']: #, '100', '1000', '10000', '100000']:
    fdf=kfpd.read_sql(sql + n,conn)
    show_timings(fdf)

fdf.head()